In [3]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import csv
from matplotlib import pyplot as plt
import seaborn as sns
import plotly 
import plotly.plotly as py 
import plotly.graph_objs as go

In [2]:
#将爬取到的信息存入BeautifulSoup里
mp1 = BeautifulSoup(open('mypage1.txt'))
mp2 = BeautifulSoup(open('mypage2.txt'))
mp3 = BeautifulSoup(open('mypage3.txt'))
mp4 = BeautifulSoup(open('mypage4.txt'))
#进行一次data cleaning得到新的csv
#（在mp4中发现有一个网页无法打开，找出来并且在原来的csv中删除，因为会对接下来的数据分析造成不便）

#找到所有的年份
def find_year(mp):
    myh1s = mp.find_all('h1')
    years = []
    for myh1 in myh1s:
        my_year = myh1.find('span',attrs = {'class':'year'})
        if my_year is not None:
            my_year = my_year.text.strip('()')
        else:
            my_year = None
        years.append(my_year)
    return years

years1 = find_year(mp1)
years2 = find_year(mp2)
years3 = find_year(mp3)
years4 = find_year(mp4)
all_years = years1 + years2 + years3 + years4

#找到所有评分了的人数
def find_rating_people(mp):
    rps = []
    rs = mp.find_all('div',attrs = {'class':'rating_sum'})
    for r in rs:
        rp = r.find('span',attrs = {'property':'v:votes'})
        if rp is not None:
            rp = rp.text.strip('人评价')
        else:
            rp = None
        rps.append(rp)
    return rps

rp1 = find_rating_people(mp1)
rp2 = find_rating_people(mp2)
rp3 = find_rating_people(mp3)
rp4 = find_rating_people(mp4)
rp = rp1 + rp2 + rp3 +rp4

#给抓出来的电视赋上排名
rank = []
for i in range(1,2175):
    rank.append((i))

#打开这个新的csv
new_table = pd.read_csv('my_TV_series_list.csv',encoding = 'gbk')

#在表格中插入年份和评价的人数
new_table.insert(0,'排名',rank)
new_table.insert(2,'年份',all_years)
new_table.insert(4,'评价人数',rp)
new_table

#发现编剧、集数、单集片长所需要的这几个信息都在info中
def all_info(mp):
    information = mp.find_all('div',attrs = {'id':'info'})
    return information

info1 = all_info(mp1)
info2 = all_info(mp2)
info3 = all_info(mp3)
info4 = all_info(mp4)

all_info = info1 + info2 + info3 + info4

#在爬集数的数据的时候发现集数的数字在span之外，但包含在info中，因此发现可以对info操作，将问题变成对字符串的操作，会更加高效和简单
information = []
for info in all_info:
    information.append(info.text.strip().split('\n'))

headers = ['编剧', '类型', '集数', '单集片长']
def needs(header):
    req = []
    for inf in information:
        a = None
        for i in inf:
            if i.find(header) != -1:
                a = i[len(header) + 1:].strip('分钟') #+1是因为还有”：”
                break
        req.append(a)
    return req
for header in headers:
    new_table[header] = needs(header)
    
#将抓取的信息放入这个表格中并进行顺序的调整
new_table.insert(6, '编剧', new_table.pop('编剧'))
new_table.insert(8, '类型', new_table.pop('类型'))
new_table.insert(9, '集数', new_table.pop('集数'))
new_table.insert(10, '单集片长', new_table.pop('单集片长'))
new_table.insert(11, '豆瓣网址', new_table.pop('豆瓣网址'))

#输出最终需要的csv
new_table.to_csv('results.csv', encoding = 'gbk',index = False)